In [4]:
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [71]:
# checing if o3d is instaleld
o3d.__version__

'0.17.0'

#### Visualise raw point cloud

In [35]:
pcd = o3d.io.read_point_cloud("./data/P001 2022-01-25 01_39_54.ply")
print(pcd)
print(np.asarray(pcd.points))
# Print min and max coordinates
print("Min:", np.min(pcd.points, axis=0))
print("Max:", np.max(pcd.points, axis=0))

# visualise raw point cloud 
# o3d.visualization.draw_geometries([pcd])

PointCloud with 247847 points.
[[-1.19145143  0.27242577  0.04408778]
 [-1.18545151  0.27242577  0.04483778]
 [-1.17945147  0.27242577  0.04483778]
 ...
 [ 1.26217365 -0.26157439  0.03658778]
 [ 1.26217365 -0.26157439  0.03658778]
 [ 1.24454856 -0.26757431  0.03396278]]
Min: [-1.26345146 -1.42557454 -0.02866222]
Max: [1.32254863 1.49005067 1.83321273]


In [72]:
print("Convert mesh to a point cloud and estimate dimensions")
armadillo = o3d.data.ArmadilloMesh()
mesh = o3d.io.read_triangle_mesh(armadillo.path)
mesh.compute_vertex_normals()

pcd = mesh.sample_points_poisson_disk(5000)
diameter = np.linalg.norm(
    np.asarray(pcd.get_max_bound()) - np.asarray(pcd.get_min_bound()))
# o3d.visualization.draw_geometries([pcd])

print("Define parameters used for hidden_point_removal")
camera = [0, 0, diameter]
radius = diameter * 100

print("Get all points that are visible from given view point")
_, pt_map = pcd.hidden_point_removal(camera, radius)

print("Visualize result")
pcd = pcd.select_by_index(pt_map)

draw_geoms_list = [pcd]
o3d.visualization.draw_geometries(draw_geoms_list)

Convert mesh to a point cloud and estimate dimensions
Define parameters used for hidden_point_removal
Get all points that are visible from given view point
Visualize result


In [98]:
x = np.linspace(-3, 3, 401)
mesh_x, mesh_y = np.meshgrid(x,x)
z = np.cos((np.power(mesh_x, 2) + np.power(mesh_y, 2))) 
z_norm = (z-z.min()) / (z.max() - z.min())
xyz = np.zeros((np.size(mesh_x), 3))
xyz[:, 0] = np.reshape(mesh_x, -1)
xyz[:, 1] = np.reshape(mesh_y, -1)
xyz[:, 2] = np.reshape(z_norm, -1)
print('xyz')
print(xyz)

# !converting np array to point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
o3d.visualization.draw_geometries([pcd])

xyz
[[-3.         -3.          0.83015834]
 [-2.985      -3.          0.79516409]
 [-2.97       -3.          0.75798531]
 ...
 [ 2.97        3.          0.75798531]
 [ 2.985       3.          0.79516409]
 [ 3.          3.          0.83015834]]


In [81]:
pcd = o3d.io.read_point_cloud("./data/P001 2022-01-25 01_39_54.ply")

# created coordinate system frame
coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=2, origin=[0,0,0]) 

# bounding box
bbox = pcd.get_axis_aligned_bounding_box()
bbox_points = np.asarray(bbox.get_box_points())

# fun


bbox_pcd = o3d.geometry.PointCloud()
bbox_pcd.points = o3d.utility.Vector3dVector(bbox_points)
o3d.visualization.draw_geometries([bbox_pcd])

bbox_points.paint_uniform_color([1,0,0])

o3d.visualization.draw_geometries([pcd, coord_frame])

AttributeError: 'numpy.ndarray' object has no attribute 'paint_uniform_color'

In [73]:
diameter = np.linalg.norm(np.asarray(pcd.get_max_bound()) - np.asarray(pcd.get_min_bound()))
print("Define parameters used for hidden_point_removal")
camera = [0, 0, diameter]
radius = diameter * 100000

print("Get all points that are visible from given view point")
_, pt_map = pcd.hidden_point_removal(camera, radius)

print("Visualize result")
pcd = pcd.select_by_index(pt_map)

draw_geoms_list = [pcd]
o3d.visualization.draw_geometries(draw_geoms_list)
# print(diameter)

Define parameters used for hidden_point_removal
Get all points that are visible from given view point
Visualize result


### Voxel downsampling

- Lidar scans can be very large with millions of points, to improve efficiency, we reduce the overall number of points in the point cloud while retaining the overall structure and shape

In [6]:
voxel_size = 0.05
downpcd = pcd.voxel_down_sample(voxel_size)
o3d.visualization.draw_geometries([downpcd])

### Vertex normal estimation

- `estimate_normals` function finds adjacent points and calculates the principal axis of the adjacent points using covariance analysis
    - `radius` = search radius, `max_nn` = maximum nearest neighbour
    - so radius = 0.1 (10cm search radius) and max_nn = 30 (only considers 30 neighbours)
- estimates normal vector (vector perpindicular to tangent plane) for each point in point cloud.
- sensitive to noise and variations & computationally expensive

In [7]:
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
o3d.visualization.draw_geometries([downpcd], point_show_normal=True)

#### Estimated vertex normal

In [8]:
downpcd.normals[0]

array([0.0088273 , 0.05764676, 0.99829802])

In [9]:
help(downpcd)

Help on PointCloud in module open3d.cpu.pybind.geometry object:

class PointCloud(Geometry3D)
 |  PointCloud class. A point cloud consists of point coordinates, and optionally point colors and point normals.
 |  
 |  Method resolution order:
 |      PointCloud
 |      Geometry3D
 |      Geometry
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __add__(...)
 |      __add__(self: open3d.cpu.pybind.geometry.PointCloud, arg0: open3d.cpu.pybind.geometry.PointCloud) -> open3d.cpu.pybind.geometry.PointCloud
 |  
 |  __copy__(...)
 |      __copy__(self: open3d.cpu.pybind.geometry.PointCloud) -> open3d.cpu.pybind.geometry.PointCloud
 |  
 |  __deepcopy__(...)
 |      __deepcopy__(self: open3d.cpu.pybind.geometry.PointCloud, arg0: dict) -> open3d.cpu.pybind.geometry.PointCloud
 |  
 |  __iadd__(...)
 |      __iadd__(self: open3d.cpu.pybind.geometry.PointCloud, arg0: open3d.cpu.pybind.geometry.PointCloud) -> open3d.cpu.pybind.geometry.Point

In [10]:
print("Normal vectors of the first 10 points")
print(np.asarray(downpcd.normals)[:10, :])

Normal vectors of the first 10 points
[[ 0.0088273   0.05764676  0.99829802]
 [ 0.15901707 -0.02367288  0.98699198]
 [ 0.69142119  0.53064376 -0.49025906]
 [-0.0245555   0.0181455   0.99953378]
 [ 0.99109051 -0.10207455 -0.08555923]
 [ 0.29101677  0.07802521  0.95353097]
 [-0.03096707 -0.0216756   0.99928535]
 [ 0.6960126  -0.41536131  0.58569739]
 [ 0.01481755 -0.03984016 -0.99909619]
 [-0.01789856 -0.03677062  0.99916343]]


### Point cloud distance

- `compute_point_cloud_distance` computes the distance from source to target point cloud (only rly works with 2 sets of PCs)

In [11]:
pcd = o3d.io.read_point_cloud("./data/P001 2022-01-25 01_39_54.ply")

In [18]:
dists = pcd.compute_point_cloud_distance(downpcd)
dists = np.asarray(dists)
# print(dists)
ind = np.where(dists > 0.01)[0]
new = pcd.select_by_index(ind)
o3d.visualization.draw_geometries_with_editing([new])

In [13]:
help(downpcd)

# getattr(downpcd, 'remove_statistical_outlier', None)

Help on PointCloud in module open3d.cpu.pybind.geometry object:

class PointCloud(Geometry3D)
 |  PointCloud class. A point cloud consists of point coordinates, and optionally point colors and point normals.
 |  
 |  Method resolution order:
 |      PointCloud
 |      Geometry3D
 |      Geometry
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __add__(...)
 |      __add__(self: open3d.cpu.pybind.geometry.PointCloud, arg0: open3d.cpu.pybind.geometry.PointCloud) -> open3d.cpu.pybind.geometry.PointCloud
 |  
 |  __copy__(...)
 |      __copy__(self: open3d.cpu.pybind.geometry.PointCloud) -> open3d.cpu.pybind.geometry.PointCloud
 |  
 |  __deepcopy__(...)
 |      __deepcopy__(self: open3d.cpu.pybind.geometry.PointCloud, arg0: dict) -> open3d.cpu.pybind.geometry.PointCloud
 |  
 |  __iadd__(...)
 |      __iadd__(self: open3d.cpu.pybind.geometry.PointCloud, arg0: open3d.cpu.pybind.geometry.PointCloud) -> open3d.cpu.pybind.geometry.Point

In [14]:
# def draw_registration_result_original_color(source, target, transformation):
#     source_temp = copy.deepcopy(source)
#     source_temp.transform(transformation)
#     o3d.visualization.draw_geometries([source_temp, target])

### bounding box

In [16]:
# aabb = downpcd.get_axis_aligned_bounding_box()
# aabb.color = (1, 0, 0)

# # Compute the oriented bounding box
# obb = downpcd.get_oriented_bounding_box()
# obb.color = (0,1,0)  # green

bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=(0.5,0.6,0.5), max_bound=(3,4,2))

# Crop the point cloud
cropped_pcd = pcd.crop(bbox)

# Visualize the cropped point cloud
o3d.visualization.draw_geometries([cropped_pcd])

# Display the point cloud with its bounding boxes
# o3d.visualization.draw_geometries([downpcd, aabb, obb])

### dbscan clustering

- grouping point clouds together, density based clustering algorithm
- `cluster_dbscan` takes `eps` = distance to neighbours in cluster, `min_points` = min number of points to form cluster
- the function returns `labels` where the label `-1` indicates noise

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(pcd.cluster_dbscan(eps = 0.02, min_points=10, print_progress=True))

max_label = labels.max()

print(f"point cloud has {max_label + 1} clusters") # index count starts from 0, so + 1 to get actual number of clusters
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
print(colors)

colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([pcd])

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 37
point cloud has 37 clusters
[[0.12156863 0.46666667 0.70588235 1.        ]
 [0.12156863 0.46666667 0.70588235 1.        ]
 [0.12156863 0.46666667 0.70588235 1.        ]
 ...
 [0.12156863 0.46666667 0.70588235 1.        ]
 [0.12156863 0.46666667 0.70588235 1.        ]
 [0.12156863 0.46666667 0.70588235 1.        ]]


### plane segmentation

- finds plane with largest support in point cloud via `segment_plane`
- `distance_threshold` = max distance a point can have to an estimated plane to be considered an inlier
- `ransac_n` = number of points that are randomly sampled to estimate a plane
- `num_iterations` = how often a random plane is sampled and verified
- Returns a list of indices of the inlier points

In [ ]:
model, inliers = pcd.segment_plane(distance_threshold=0.02, ransac_n=5, num_iterations=1000)
[a, b, c, d] = model

print(f"{a}x + {b}y + {c}z + {d} = 0")

inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1,0,0])
outlier_cloud = pcd.select_by_index(inliers, invert=True)
o3d.visualization.draw_geometries([outlier_cloud])

-0.004968237198999695x + -0.003690443544188644y + 0.9999808484393997z + -0.0358404879681415 = 0
